# **Caso 2: Comparación de Direcciones**
## **Entendimiento del Problema y de los Datos**


### Creamos la conexión con Vantage

In [1]:
%connect DataAnalyst

Password: ·······


Success: 'DataAnalyst' connection established and activated for user 'pocuser', with default database 'DW_LANDING'


### Exploramos los Datos

In [2]:
-- Primeros registros
SEL TOP 10 * FROM caso2;

,COL_00,COL_01,COL_02,COL_03,COL_04
1,,18507577,250105,MANZANA JORGE IÑAPARI 202,202
2,,13999634,250107,OVALO FRANCISCO C.TELLO C20,C20
3,,3315076,250101,AVENIDA SALVADOR VARGAS 1-B,1-B
4,,9491691,250107,OVALO AMADOR REYES 404,404
5,,4983748,250107,AV ANDRES IÑAPARI 001,001
6,,11160363,250101,AV SALVADOR BENAVIDES A01,A01
7,,15668306,250107,OVALO AMADOR CONCEPCIÓN 8981,8981
8,,7823019,250101,CARRETERA LUIS JOSÉ 2-01,2-01
9,,1646404,250102,AVENIDA FRANCISCO VARGAS 9,9
10,,12330962,250101,CALLE JUAN CARLOS MÜLLER 20,20


### Tamaño de la tabla y cantidad de direcciones distintas

In [3]:
-- Cantidad de Registros
SEL COUNT(1) N, COUNT(DISTINCT COL_03) NDIR FROM caso2;

,N,NDIR
1,20000000,15750


### Funciones de Reemplazo de Texto

In [4]:
SELECT TOP 20 REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(
REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(
REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(REGEXP_REPLACE(
REGEXP_REPLACE(REGEXP_REPLACE(COL_03 ,'BLOCK' ,''),'ASC VIV',''),' DE VIV.',''),'VIV.',''),'A.H.',''),'AGRUP.',''),'C.P.',''),
'AGRP.',''),'VIV ',''),'VEN ' ,''),'AMPL.',''),'HAB.',''),'ASC VIV',''),'C.HABITACIONAL',''),'AMP ',''),'AMPLC.',''),'A.H.',''),
'PROG.',''),'SCTR ',''),'P. JOVEN ',''),'PASAJE ',''),'DPTO.',''),'APV.',''),'BL. ',''),'BK. ','') COL03 FROM CASO2;

,COL03
1,AVENIDA JORGE JOSÉ 8981
2,MANZANA JORGE BENAVIDES 100
3,JIRON MANUEL ÑAÑA 333
4,CALLE ANDRES CONIÓN 2-01
5,AV MANUEL CONIÓN 8981
6,MANZANA AMADOR MÜLLER 2-01
7,AVENIDA MANUEL IÑAPARI 9
8,JIRON LUIS JOSÉ 320
9,CALLE FRANCISCO VARGAS 001
10,JIRON RENATO MÜLLER C20


### Creando la tabla con direcciones únicas

In [5]:
CREATE TABLE CASO2RED AS (
    SELECT MIN(COL_01) COL_01, COL_03 FROM CASO2 GROUP BY 2) WITH DATA 
PRIMARY INDEX(COL_01);

Success: 0 rows affected

### Primeros Registros

In [6]:
SELECT TOP 10 * FROM CASO2RED;

,COL_01,COL_03
1,4690,JIRON AMADOR CONCEPCIÓN 1001
2,29629,CARRETERA ANDRES VARGAS 8981
3,13397,CALLE AMADOR BENAVIDES 1001
4,183,MANZANA ANDRES CONCEPCIÓN 320
5,17883,MANZANA SALVADOR OCTUBRE A01
6,20901,MANZANA ANDRES PULPA 20
7,2997,MANZANA LUIS JOSÉ 2A
8,5485,CALLE JUAN CARLOS JOSÉ 2-01
9,1672,CALLE LUIS MÜLLER 20
10,18108,CALLE FRANCISCO BOLOGNESI A01


### Cantidad de Registros

In [7]:
SELECT COUNT(1) N FROM CASO2RED;

,N
1,15750


![Slide](images/Diapositiva5.PNG)

## **Creando la Matriz para comparar todas las direcciones**

In [8]:
CREATE TABLE CASO2CRUCE AS (
SELECT nom1.COL_01 userid, nom2.COL_01 userid2, nom1.COL_03 name1, nom2.COL_03 name2
FROM CASO2RED nom1, (SELECT * FROM CASO2RED WHERE COL_01<3000) nom2
WHERE userid <> USERID2
) WITH DATA PRIMARY INDEX(USERID);

Success: 0 rows affected

### Cantidad de Registros

In [9]:
SELECT COUNT(1) N FROM CASO2CRUCE;

,N
1,42443555


In [10]:
SELECT TOP 10 * FROM CASO2CRUCE;

,userid,userid2,name1,name2
1,15885,1846,JIRON ENRIQUE OCTUBRE 2A,CARRETERA JORGE JOSÉ 8981
2,15885,990,JIRON ENRIQUE OCTUBRE 2A,OVALO SALVADOR ZELA 1001
3,15885,2052,JIRON ENRIQUE OCTUBRE 2A,MANZANA MANUEL ENERO A01
4,15885,33,JIRON ENRIQUE OCTUBRE 2A,JIRON ANDRES IÑAPARI 001
5,15885,2113,JIRON ENRIQUE OCTUBRE 2A,OVALO FRANCISCO OCTUBRE 9
6,15885,176,JIRON ENRIQUE OCTUBRE 2A,JIRON MANUEL VARGAS 404
7,15885,1032,JIRON ENRIQUE OCTUBRE 2A,CALLE JUAN CARLOS ENERO 1-B
8,15885,235,JIRON ENRIQUE OCTUBRE 2A,AVENIDA RENATO IÑAPARI 9
9,15885,1562,JIRON ENRIQUE OCTUBRE 2A,MANZANA RENATO CONCEPCIÓN C20
10,15885,704,JIRON ENRIQUE OCTUBRE 2A,AVENIDA FRANCISCO DE LA CRUZ 333


### Creando la Tabla Final de Direcciones más similares (Distancia de Levenshtein)

In [11]:
CREATE TABLE CASO2COMPARA AS (
SELECT * FROM (
SELECT * FROM StringSimilarity (
 ON CASO2CRUCE PARTITION BY ANY
 USING
 ComparisonColumnPairs ('LD(name1,name2) AS similarity')
 Accumulate ('userid','userid2')
) AS dt) as dt1
QUALIFY ROW_NUMBER() OVER(PARTITION BY userid ORDER BY similarity DESC) = 1) WITH DATA;

Success: 0 rows affected

### Primeros Registros

In [12]:
SELECT TOP 10 * FROM CASO2COMPARA;

,userid,userid2,similarity
1,2814,821,0.8333333333333334
2,3079,2063,0.9047619047619048
3,82220,1871,0.875
4,11929,2083,0.7931034482758621
5,60543,1610,0.9523809523809523
6,12112,2541,0.9230769230769231
7,12928,2514,0.9375
8,17435,2078,0.9545454545454546
9,50368,2263,0.9130434782608695
10,22064,102,0.9565217391304348


### Revisando algunos ejemplos

In [13]:
SELECT * FROM CASO2RED WHERE COL_01=2814;
SELECT * FROM CASO2RED WHERE COL_01=821;

,COL_01,COL_03
1,2814,AV AMADOR ZELA 1-B


,COL_01,COL_03
1,821,AV AMADOR ZELA C20


In [14]:
SELECT * FROM CASO2RED WHERE COL_01=60543;
SELECT * FROM CASO2RED WHERE COL_01=1610;

,COL_01,COL_03
1,60543,AV AMADOR OCTUBRE C20


,COL_01,COL_03
1,1610,AV AMADOR OCTUBRE 20


In [15]:
SELECT * FROM CASO2RED WHERE COL_01=22;
SELECT * FROM CASO2RED WHERE COL_01=4458;

,COL_01,COL_03
1,22,CARRETERA JUAN CARLOS DE LA CRUZ 001


,COL_01,COL_03
1,4458,CARRETERA JUAN CARLOS DE LA CRUZ A01


## **Desarrollando Análisis más Enfocados en la Ubicación**
### Distribución por Ubigeo

In [16]:
SEL COL_02, COUNT(1) N, COUNT(DISTINCT COL_03) NDIR FROM caso2 GROUP BY 1;

,COL_02,N,NDIR
1,250102,3999670,15750
2,250101,4002458,15750
3,250105,4001648,15750
4,250201,3997855,15750
5,250107,3998369,15750


### Si quisiéramos reducir el tamaño de las matrices, se podría hacer una búsqueda por UBIGEO

In [17]:
SELECT TOP 10 * FROM (
SELECT MIN(COL_01) COL_01, COL_03 FROM CASO2 
WHERE COL_02='250102'
GROUP BY 2) as DT;

,COL_01,COL_03
1,53150,JIRON RENATO MÜLLER A01
2,60431,AVENIDA MANUEL PULPA 8981
3,55928,MANZANA ENRIQUE PULPA 202
4,102012,JIRON FRANCISCO ENERO 404
5,84405,CALLE SALVADOR PULPA 1-B
6,30253,AV RENATO OCTUBRE 202
7,208511,AVENIDA JUAN CARLOS OCTUBRE A01
8,147848,CARRETERA RENATO ENERO 001
9,118362,OVALO AMADOR ZELA 202
10,88616,AVENIDA JUAN CARLOS JOSÉ 1001


### Distancia Coseno (Se recomienda para comparar nombres en desorden)

In [18]:
SELECT * FROM StringSimilarity (
 ON (SELECT TOP 10 * FROM CASO2CRUCE) PARTITION BY ANY
 USING
 ComparisonColumnPairs ('cosine(name1,name2) AS similarity')
 Accumulate ('userid','userid2','name1','name2')
) AS dt

,userid,userid2,name1,name2,similarity
1,22064,1493,MANZANA JORGE REYES 20,JIRON MANUEL MÜLLER 202,0.7416198487095663
2,22064,1472,MANZANA JORGE REYES 20,AVENIDA RENATO OCTUBRE 333,0.7324220137800286
3,22064,1737,MANZANA JORGE REYES 20,CARRETERA RENATO DE LA CRUZ 8981,0.7911329000851551
4,22064,1207,MANZANA JORGE REYES 20,AVENIDA ENRIQUE ÑAÑA 1001,0.7458152383799166
5,22064,921,MANZANA JORGE REYES 20,OVALO ANDRES REYES 404,0.8181818181818182
6,22064,1144,MANZANA JORGE REYES 20,CALLE LUIS JOSÉ 202,0.5606119105813882
7,22064,2084,MANZANA JORGE REYES 20,MANZANA JORGE PULPA 1001,0.7888426505157585
8,22064,1636,MANZANA JORGE REYES 20,AVENIDA JORGE BOLOGNESI 2A,0.8259685381260148
9,22064,2431,MANZANA JORGE REYES 20,MANZANA MANUEL BENAVIDES 20,0.829296991057688
10,22064,290,MANZANA JORGE REYES 20,CARRETERA ENRIQUE VARGAS 100,0.8258948315174716


### Distancia de Jaccard (Una alternativa intetesante para las Direcciones porque considera co-ocurrencias)

In [19]:
SELECT * FROM StringSimilarity (
 ON (SELECT TOP 10 * FROM CASO2CRUCE) PARTITION BY ANY
 USING
 ComparisonColumnPairs ('jaccard(name1,name2) AS similarity')
 Accumulate ('userid','userid2','name1','name2')
) AS dt

,userid,userid2,name1,name2,similarity
1,7239,2782,CALLE RENATO JOSÉ 2A,AV SALVADOR BENAVIDES 333,0.4444444444444444
2,7239,2025,CALLE RENATO JOSÉ 2A,CALLE ENRIQUE JOSÉ 404,0.6111111111111112
3,7239,923,CALLE RENATO JOSÉ 2A,MANZANA ENRIQUE MÜLLER 202,0.35
4,7239,1209,CALLE RENATO JOSÉ 2A,CALLE ANDRES REYES C20,0.5625
5,7239,2269,CALLE RENATO JOSÉ 2A,OVALO JORGE JOSÉ 404,0.5294117647058824
6,7239,1493,CALLE RENATO JOSÉ 2A,JIRON MANUEL MÜLLER 202,0.5
7,7239,719,CALLE RENATO JOSÉ 2A,OVALO SALVADOR PULPA 8981,0.3
8,7239,271,CALLE RENATO JOSÉ 2A,AVENIDA ANDRES BENAVIDES 2A,0.4117647058823529
9,7239,88,CALLE RENATO JOSÉ 2A,OVALO ENRIQUE CONCEPCIÓN 9,0.4
10,7239,519,CALLE RENATO JOSÉ 2A,AVENIDA JUAN CARLOS VARGAS A01,0.5


### Distancia de Jaro-Winkler (Considera el inicio de cada palabra)

In [20]:
SELECT * FROM StringSimilarity (
 ON (SELECT TOP 10 * FROM CASO2CRUCE) PARTITION BY ANY
 USING
 ComparisonColumnPairs ('jaro_winkler(name1,name2) AS similarity')
 Accumulate ('userid','userid2','name1','name2')
) AS dt

,userid,userid2,name1,name2,similarity
1,22064,1493,MANZANA JORGE REYES 20,JIRON MANUEL MÜLLER 202,0.6891084760649978
2,22064,1472,MANZANA JORGE REYES 20,AVENIDA RENATO OCTUBRE 333,0.605894105894106
3,22064,1737,MANZANA JORGE REYES 20,CARRETERA RENATO DE LA CRUZ 8981,0.5375145687645687
4,22064,1207,MANZANA JORGE REYES 20,AVENIDA ENRIQUE ÑAÑA 1001,0.5754312354312354
5,22064,921,MANZANA JORGE REYES 20,OVALO ANDRES REYES 404,0.6767676767676768
6,22064,1144,MANZANA JORGE REYES 20,CALLE LUIS JOSÉ 202,0.5602870813397128
7,22064,2084,MANZANA JORGE REYES 20,MANZANA JORGE PULPA 1001,0.8787878787878788
8,22064,1636,MANZANA JORGE REYES 20,AVENIDA JORGE BOLOGNESI 2A,0.6418026418026418
9,22064,2431,MANZANA JORGE REYES 20,MANZANA MANUEL BENAVIDES 20,0.8639730639730641
10,22064,290,MANZANA JORGE REYES 20,CARRETERA ENRIQUE VARGAS 100,0.6280663780663781


### Limpiando las tablas creadas

In [21]:
DROP TABLE CASO2CRUCE;

Success: 23 rows affected

In [22]:
DROP TABLE CASO2RED;

Success: 20 rows affected

In [23]:
DROP TABLE CASO2COMPARA;

Success: 21 rows affected

Copyright 2023. Elaborado por Luis Cajachahua bajo licencia MIT